# 連立１次方程式の数値解法

## [`\`演算子](https://docs.julialang.org/en/v1/stdlib/LinearAlgebra/#Base.:\\-Tuple{AbstractMatrix,%20AbstractVecOrMat})による数値解の計算
$n$次正方行列 $A$, $n$項列ベクトル $b$ に対して，
連立一次方程式 $A\vec{x} = \vec{b}$ の解は `A\b` で計算できる．

In [1]:
A = [1 1 1; 1 2 3; 1 4 9]
b = [1, 2, 3]
A \ b

3-element Vector{Float64}:
 -0.5000000000000002
  2.0000000000000004
 -0.5000000000000002

出力された数値解は厳密解 $\vec{x} = (-1/2, 2, -1/2)^T$ とほぼ一致している．  
仮数部の末尾が0でないのは丸め誤差の影響である．

念のため，行列式とランクを確かめておこう．

In [2]:
using LinearAlgebra
@show det(A) rank(A);

det(A) = 1.9999999999999996
rank(A) = 3


<div class="alert alert-warning"> 
Warning: 
連立１次方程式の解は inv(A)*b でも得られるが，A\b よりも効率が悪いので使われない．
</div>

## 悪条件な連立１次方程式
連立１次方程式によっては，丸め誤差が増幅されて数値解が厳密解と大きく異なる場合がある．  
そのような連立１次方程式（の係数行列）を悪条件であるという．

$n$次Hilbert行列 $H = (h_{ij})$：
$$
 h_{ij} := \frac{1}{i+j-1}, \qquad 
 1 \le i, j \le n.
$$
は $n$が大きいと，悪条件であることが知られている．

連立一次方程式
$$
  H \vec{x} = \vec{f} := \begin{pmatrix} 1 \\ 1 \\ \vdots \\ 1 
\end{pmatrix}
$$
を`\`演算子で数値解を計算してみよう．

数値解の誤差を確認するために先に厳密解を求める．  
Hilbert行列の逆行列の第$(i,j)$成分は整数値であり，
$$
  (-1)^{i+j}(i+j-1){\binom {n+i-1}{n-j}}{\binom {n+j-1}{n-i}}{\binom {i+j-2}{i-1}}^{2}
$$
と書き表せる．ここで，$\binom{p}{q}$ は二項係数である．
（参考：[Wikipedia/HilbertMatrix](https://en.wikipedia.org/wiki/Hilbert_matrix)）



In [3]:
n = 16
# Hilbert行列の逆行列．整数型で定義する．
hilbert_inv(n) = Int64[(-1)^(i+j)*(i+j-1)*binomial(n+i-1,n-j)*binomial(n+j-1,n-i)*binomial(i+j-2,i-1)^2 for i in 1:n, j in 1:n]

f(n) = ones(Int64, n) # 右辺のベクトル
@show exact_x = hilbert_inv(n)*f(n) #厳密解

exact_x = hilbert_inv(n) * f(n) = [-16, 4080, -257040, 7054320, -105814800, 977728752, -5975009040, 25241364720, -75724094160, 163601438000, -255218243280, 284747626800, -221470376400, 114011377200, -34901442000, 4808643120]


16-element Vector{Int64}:
           -16
          4080
       -257040
       7054320
    -105814800
     977728752
   -5975009040
   25241364720
  -75724094160
  163601438000
 -255218243280
  284747626800
 -221470376400
  114011377200
  -34901442000
    4808643120

これは整数演算のみで計算され，overflowも発生しないため，厳密解と一致していると考えてよい．  

`\`演算子による数値解の計算は，各自の手で確かめてください．

In [4]:
H(n) = Float64[1/(i+j-1) for i in 1:n, j in 1:n] # Hilbert行列
f(n) = ones(Float64, n) 

# 数値解の計算
# x = H(16)\f(16) 

# 誤差の表示
# @show exact_x - x

f (generic function with 1 method)

## 係数行列がフルランクでない場合
簡約階段行列に変形して不定性も含めて求める．  
簡約階段行列の計算には`RowEchelon`パッケージの `rref()`を使うので，
あらかじめインストールしておく．  
（簡約階段の形状のことを英語で reduced echelon form という．）
```
pkg> add RowEchelon
```

次の係数行列に対して，$B\vec{x} = \vec{d}$ を解くことを考える．

In [5]:
B = [1 3 -2 4;
    2 -1 3 0;
    8 3 5 8;
    7 -7 14 -4]

4×4 Matrix{Int64}:
 1   3  -2   4
 2  -1   3   0
 8   3   5   8
 7  -7  14  -4

In [6]:
@show rank(B) # ランクの確認

rank(B) = 2


2

In [7]:
# rref() で Bの簡約階段行列を計算する．
using RowEchelon
C = rref(B)

4×4 Matrix{Float64}:
 1.0  0.0   1.0  0.571429
 0.0  1.0  -1.0  1.14286
 0.0  0.0   0.0  0.0
 0.0  0.0   0.0  0.0

`B` の kernelの基底を計算する．$ \vec{v} = (v_1, v_2, v_3, v_4) \in \ker B$は
$$
   C \vec{v} = \vec{0}
$$
と同値である．$(v_3,v_4) = (1,0), (0,1)$に対して それぞれ $(v_1, v_2)$
を求めれば，$\ker B$ の基底ベクトルが得られる．

したがって，次の方程式の解 $\vec{v} = (v_i), \vec{w} = (w_i)$が基底ベクトルになる．
$$
\begin{pmatrix}
1 & 0 & * & * \\
0 & 1 & * & * \\
0 & 0 & 1 & 0 \\ 
0 & 0 & 0 & 1 
\end{pmatrix}
\begin{pmatrix}v_1 & w_1 \\ v_2 & w_2 \\ v_3 & w_3 \\ v_4 & w_4 \end{pmatrix}
= 
\begin{pmatrix} 0 & 0 \\ 0 & 0 \\ 1 & 0 \\ 0 &  1\end{pmatrix}
$$

In [8]:
C[3, 3] = C[4, 4] = 1
@show C
v = C \ [0 0; 0 0; 1 0; 0 1]

C = [1.0 0.0 1.0 0.5714285714285714; 0.0 1.0 -1.0 1.1428571428571428; 0.0 0.0 1.0 0.0; 0.0 0.0 0.0 1.0]


4×2 Matrix{Float64}:
 -1.0  -0.571429
  1.0  -1.14286
  1.0   0.0
  0.0   1.0

In [9]:
# 確認
@show B * v[:, 1] B * v[:, 2];

B * v[:, 1] = [0.0, 0.0, 0.0, 0.0]
B * v[:, 2] = [0.0, 0.0, 0.0, 0.0]


### (i) $\vec{d} = (6, 4, 24, 10)$ の場合
拡大係数行列 $(B | \vec{d})$のランクが 2 $(= \mathrm{rank} B)$なので，（不定性を含んだ）一般解が存在する．

In [10]:
d = [6, 4, 24, 10]  # = B*[1,1,1,1]
@show rank([B d])   # 拡大係数行列のランク
@show x = B \ d

rank([B d]) = 2
x = B \ d = [3.7142857142857144, 6.344131569286608e-17, -1.1428571428571428, 5.551115123125783e-17]


4-element Vector{Float64}:
  3.7142857142857144
  6.344131569286608e-17
 -1.1428571428571428
  5.551115123125783e-17

これに，先に求めた $\ker B$ を加えたものが一般解となる．

### (ii) $\vec{d} = (1,2,3,4)$の場合
この場合は $\mathrm{rank} (B | \vec{d}) = 3 > 2 = \mathrm{rank} B$ となるので解は存在しない．

もし，これを知らずに解を計算するとどうなるか試してみる．

In [11]:
d = [1, 2, 3, 4]
@show rank([B d])  # 拡大係数行列のランク
@show x = B \ d    # 何らかの数値は出力される 
@show B * x
@show B * x == d

rank([B d]) = 3
x = B \ d = [-1.470563143631182e16, -2.058788401083655e16, 2.9411262872623645e15, 2.058788401083655e16]
B * x = [0.0, 6.0, 0.0, 32.0]
B * x == d = false


false